In [69]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from bleak import BleakClient

address = "0D0F7D44-48A6-71E5-4523-A5427505D4C1"  # Replace with your WT901BLE's address

async def show_characteristics():
    async with BleakClient(address) as client:
        await client.get_services()  # still needed to populate `client.services`
        for service in client.services:
            print(f"[Service] {service.uuid}")
            for char in service.characteristics:
                print(f"  [Characteristic] {char.uuid} — {char.properties}")

await show_characteristics()

from bleak import BleakScanner
import asyncio

async def scan():
    devices = await BleakScanner.discover()
    for d in devices:
        print(f"{d.name} — {d.address}")

asyncio.run(scan())


/var/folders/y_/gz8q6hf17y1dy5vbgdbyc7ch0000gn/T/ipykernel_60393/2816786376.py:11: FutureWarning: This method will be removed future version, use the services property instead.
  await client.get_services()  # still needed to populate `client.services`


[Service] 0000ffe5-0000-1000-8000-00805f9a34fb
  [Characteristic] 0000ffe9-0000-1000-8000-00805f9a34fb — ['write-without-response', 'write']
  [Characteristic] 0000ffe4-0000-1000-8000-00805f9a34fb — ['notify']
None — 1893BA7D-A1DD-9CC7-2C9B-B8CBE5E07E7E
None — 1343958F-2C90-9D08-12A2-A677AE4B4D00
None — 1C94942A-8539-091F-C608-263D46FEFE1A
None — E98A2247-AAAC-4DBF-C6A4-856A58FC1631
None — 76FF91ED-4B4F-8DE5-7B1D-6ADF3B4B5D1E
None — 614F74EC-5A5C-B61E-A6AE-9716D37238F0
None — 5267102E-8C77-D240-E9BC-B5ACBECE2A72
None — 4F61E4B7-62D2-CD48-D31A-70F0A9E235ED
None — B4E785DC-3E94-C31B-1725-246B5BF3007A
None — C5152808-124B-FA01-0EB9-C357EA7D0869
None — F42C31DF-0549-872B-D657-1AC026A28C55
None — 13E79EBC-1211-FE3F-5BB8-5479DEE71752
None — 6E294DE6-82BA-F077-7328-EB0052778D5B
None — 347505D1-8192-5034-16B7-48C32B77A69A
None — 7AE1F128-816E-0D77-E441-067CBDA1115E
None — 5309D7AF-6557-1D53-CAE2-DD8DDDD1B23A
WF-C510-GFP — 88035CC9-E4C4-65C6-4AE6-0F911631A421
None — 95E8F107-CF8C-A854-744F-B604

## Plotting


In [70]:
%matplotlib qt

In [82]:
import asyncio
from bleak import BleakClient
from datetime import datetime
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
from collections import deque
import time

# BLE UUIDs
NOTIFY_UUID = "0000ffe4-0000-1000-8000-00805f9a34fb"
WRITE_UUID = "0000ffe9-0000-1000-8000-00805f9a34fb"

# Device addresses
NECK_ADDRESS = "0CE86DC8-97A4-3492-9C5B-D06CE921BC65"
PELVIS_ADDRESS = "0D0F7D44-48A6-71E5-4523-A5427505D4C1"

sample = 5000
mag_calibragtion = False

# Real-time plotting
plt.ion()
lean_history = deque(maxlen=sample)
time_history = deque(maxlen=sample)
fig, plot_ax = plt.subplots()
line, = plot_ax.plot([], [], lw=2)
plot_ax.set_ylim(-120, 180)
plot_ax.set_xlim(0, sample)
plot_ax.set_xlabel("Time (frames)")
plot_ax.set_ylabel("Lean Angle (°)")
plot_ax.set_title("Real-Time Forward Lean (Neck - Pelvis)")
plt.show(block=False)

# Data storage
raw_neck = []
raw_pelvis = []
data_neck = []
data_pelvis = []
angle_diff_data = []
latest_pitch = {"neck": None, "pelvis": None}
last_update_time = 0  # Global throttle

# Universal decoder
def decode_packet(data, label):
    global last_update_time
    now = time.time()
    if now - last_update_time < 0.1:  # Limit to 10Hz
        return
    last_update_time = now

    if data[0] != 0x55 or data[1] not in [0x61, 0x71]:
        return

    decoded = [int.from_bytes(data[i:i+2], byteorder='little', signed=True) for i in range(2, len(data), 2)]
    if len(decoded) < 9:
        return

    acc_x = decoded[0] / 32768.0 * 16 * 9.8
    acc_y = decoded[1] / 32768.0 * 16 * 9.8
    acc_z = decoded[2] / 32768.0 * 16 * 9.8
    wx = decoded[3] / 32768.0 * 2000
    wy = decoded[4] / 32768.0 * 2000
    wz = decoded[5] / 32768.0 * 2000
    roll = decoded[6] / 32768.0 * 180
    pitch = decoded[7] / 32768.0 * 180
    yaw = decoded[8] / 32768.0 * 180

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    row = {
        "timestamp": timestamp,
        "ax (m/s^2)": acc_x,
        "ay (m/s^2)": acc_y,
        "az (m/s^2)": acc_z,
        "wx (deg/s)": wx,
        "wy (deg/s)": wy,
        "wz (deg/s)": wz,
        "roll (deg)": roll,
        "pitch (deg)": pitch,
        "yaw (deg)": yaw
    }

    latest_pitch[label] = pitch

    if label == "neck":
        raw_neck.append(data.hex())
        data_neck.append(row)
    else:
        raw_pelvis.append(data.hex())
        data_pelvis.append(row)

    if latest_pitch["neck"] is not None and latest_pitch["pelvis"] is not None:
        angle_diff = abs(latest_pitch["neck"] - latest_pitch["pelvis"])
        angle_diff_data.append({"timestamp": timestamp, "lean angle (neck - pelvis)": angle_diff})
        lean_history.append(angle_diff)
        time_history.append(len(time_history))
        line.set_data(time_history, lean_history)
        plot_ax.set_xlim(max(0, len(time_history)-2000), len(time_history))
        plot_ax.figure.canvas.draw()
        plot_ax.figure.canvas.flush_events()

# Notification handlers
def handler_neck(sender, data):
    decode_packet(data, "neck")

def handler_pelvis(sender, data):
    decode_packet(data, "pelvis")

# BLE task
async def run_dual_ble():
    async with BleakClient(NECK_ADDRESS) as neck, BleakClient(PELVIS_ADDRESS) as pelvis:
        print("• Connected to NECK sensor")
        print("• Connected to PELVIS sensor")

        # === Set hardware sampling rate to 10Hz ===
        await neck.write_gatt_char(WRITE_UUID, b'\xFF\xAA\x03\x0A\x00')
        await pelvis.write_gatt_char(WRITE_UUID, b'\xFF\xAA\x03\x0A\x00')

        print("••Place the Sensor")
        await asyncio.sleep(10)
        await neck.start_notify(NOTIFY_UUID, handler_neck)
        await pelvis.start_notify(NOTIFY_UUID, handler_pelvis)

        print("Receiving data from both sensors...")
        print("••••••••• This is Reference Stand Straight •••••••••")

        await asyncio.sleep(5)
        print("Okay You are all set.")
        print("▶️ Press ENTER anytime to stop recording...")

        stop_flag = asyncio.Event()

        async def wait_for_stop():
            await asyncio.to_thread(input)
            stop_flag.set()

        asyncio.create_task(wait_for_stop())

        while not stop_flag.is_set():
            await asyncio.sleep(0.5)

        await neck.stop_notify(NOTIFY_UUID)
        await pelvis.stop_notify(NOTIFY_UUID)
        print("📴 Stopped both sensors.")

# Run it
asyncio.run(run_dual_ble())


• Connected to NECK sensor
• Connected to PELVIS sensor
••Place the Sensor
Receiving data from both sensors...
••••••••• This is Reference Stand Straight •••••••••
Okay You are all set.
▶️ Press ENTER anytime to stop recording...


AttributeError: 'NoneType' object has no attribute 'get_characteristic'

In [72]:
import os
import pandas as pd
from datetime import datetime

# === Define base save directory ===
base_dir = "/Users/moon/Documents/Posture/Bluetooth_dual_data"

# === Generate timestamped folder name ===
start_time = data_neck[0]["timestamp"]
end_time = data_neck[-1]["timestamp"]

# Format to [HH-MM_HH-MM] and folder name like: 2025-05-27_[14-05_14-35]
start_dt = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S.%f")
end_dt = datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S.%f")
folder_name = f"{start_dt.date()}_[{start_dt.strftime('%H-%M')}_{end_dt.strftime('%H-%M')}]"

# === Final output path ===
output_dir = os.path.join(base_dir, folder_name)
os.makedirs(output_dir, exist_ok=True)

# === Save files ===
pd.DataFrame(data_neck).to_csv(os.path.join(output_dir, "neck_data.csv"), index=False)
pd.DataFrame(data_pelvis).to_csv(os.path.join(output_dir, "pelvis_data.csv"), index=False)
pd.DataFrame({"raw": raw_neck}).to_csv(os.path.join(output_dir, "neck_raw.csv"), index=False)
pd.DataFrame({"raw": raw_pelvis}).to_csv(os.path.join(output_dir, "pelvis_raw.csv"), index=False)
pd.DataFrame(angle_diff_data).to_csv(os.path.join(output_dir, "lean_angle_diff.csv"), index=False)

print(f"✅ CSV files saved in: {output_dir}")



✅ CSV files saved in: /Users/moon/Documents/Posture/Bluetooth_dual_data/2025-05-30_[10-14_10-16]


## Data Anlaysis


In [74]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# === Load Data ===
neck_df = pd.read_csv(os.path.join(output_dir, "neck_data.csv"))
pelvis_df = pd.read_csv(os.path.join(output_dir, "pelvis_data.csv"))

# === Convert timestamp to datetime ===
neck_df['timestamp'] = pd.to_datetime(neck_df['timestamp'])
pelvis_df['timestamp'] = pd.to_datetime(pelvis_df['timestamp'])
# Rename first
neck_df = neck_df.rename(columns={"pitch (deg)": "neck_pitch"})
pelvis_df = pelvis_df.rename(columns={"pitch (deg)": "pelvis_pitch"})

# Then resample
neck_pitch = neck_df.set_index('timestamp').resample('100ms').mean().interpolate().reset_index()
pelvis_pitch = pelvis_df.set_index('timestamp').resample('100ms').mean().interpolate().reset_index()

# === Merge on exact resampled timestamps ===
angles_df = pd.merge(neck_pitch, pelvis_pitch, on='timestamp', how='inner')

# === Compute relative time ===
angles_df['seconds'] = (angles_df['timestamp'] - angles_df['timestamp'].min()).dt.total_seconds()

# === Use first 10 seconds as baseline posture ===
ref_df = angles_df[angles_df['seconds'] <= 10]
neck_ref = ref_df['neck_pitch'].mean()
pelvis_ref = ref_df['pelvis_pitch'].mean()

# === Normalize pitch angles ===
angles_df['rel_neck_pitch'] = angles_df['neck_pitch'] - neck_ref
angles_df['rel_pelvis_pitch'] = angles_df['pelvis_pitch'] - pelvis_ref
angles_df['lean_angle'] = angles_df['rel_neck_pitch'] - angles_df['rel_pelvis_pitch']

# === Flag slouched posture (>10° deviation) ===
angles_df['is_slouched'] = angles_df['lean_angle'].abs() > 10

# === Identify slouch blocks ≥ 60 seconds ===
log_entries = []
min_block_duration = 60  # seconds      <===================================60
block_start_idx = None

for i, is_slouched in enumerate(angles_df['is_slouched']):
    if is_slouched and block_start_idx is None:
        block_start_idx = i
    elif not is_slouched and block_start_idx is not None:
        block = angles_df.iloc[block_start_idx:i]
        duration_sec = (block['timestamp'].iloc[-1] - block['timestamp'].iloc[0]).total_seconds()
        if duration_sec >= min_block_duration:
            log_entries.append({
                "start_time": block.iloc[0]['timestamp'],
                "end_time": block.iloc[-1]['timestamp'],
                "duration_sec": duration_sec,
                "max_lean_angle": block['lean_angle'].abs().max()
            })
        block_start_idx = None

# Handle unfinished block at the end
if block_start_idx is not None:
    block = angles_df.iloc[block_start_idx:]
    duration_sec = (block['timestamp'].iloc[-1] - block['timestamp'].iloc[0]).total_seconds()
    if duration_sec >= min_block_duration:
        log_entries.append({
            "start_time": block.iloc[0]['timestamp'],
            "end_time": block.iloc[-1]['timestamp'],
            "duration_sec": duration_sec,
            "max_lean_angle": block['lean_angle'].abs().max()
        })

# === Plot Lean Angle Over Time ===
quarter_length = angles_df['seconds'].max() / 4
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(angles_df['seconds'], angles_df['lean_angle'], label='Normalized Lean Angle', color='blue')
ax.axhline(y=10, color='red', linestyle=':', label='Slouch Threshold (+10°)')
ax.axhline(y=-10, color='red', linestyle=':')

for q in [1, 2, 3]:
    ax.axvline(x=q * quarter_length, color='gray', linestyle='--')

ax.set_title("Normalized Lean Angle Over Time (Relative to First 10s)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Lean Angle (°)")
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()

# === Save Log to CSV ===
slouch_log_df = pd.DataFrame(log_entries)
log_path = os.path.join(output_dir, "angle_deviation_log.csv")
slouch_log_df.to_csv(log_path, index=False)
print("✅ Saved to:", log_path)

# === Output Summary ===
slouch_log_df


✅ Saved to: /Users/moon/Documents/Posture/Bluetooth_dual_data/2025-05-30_[10-14_10-16]/angle_deviation_log.csv


,start_time,end_time,duration_sec,max_lean_angle
0,2025-05-30 10:15:39.800,2025-05-30 10:16:55.800,76.0,79.30427


In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# === Ensure timestamp columns are datetime ===
slouch_log_df['start_time'] = pd.to_datetime(slouch_log_df['start_time'])
slouch_log_df['end_time'] = pd.to_datetime(slouch_log_df['end_time'])

# === Load neck data to compute total session duration and quarter length ===
neck_df = pd.read_csv(os.path.join(output_dir, "neck_data.csv"))
neck_df['timestamp'] = pd.to_datetime(neck_df['timestamp'])
start_time = neck_df['timestamp'].min()
end_time = neck_df['timestamp'].max()
total_duration = (end_time - start_time).total_seconds()
quarter_length = total_duration / 4

# === Assign quarters to slouch events ===
def assign_quarter(ts):
    seconds = (ts - start_time).total_seconds()
    if seconds <= quarter_length:
        return "Q1"
    elif seconds <= 2 * quarter_length:
        return "Q2"
    elif seconds <= 3 * quarter_length:
        return "Q3"
    elif seconds <= 4 * quarter_length:
        return "Q4"
    else:
        return "Extra"

slouch_log_df['quarter'] = slouch_log_df['start_time'].apply(assign_quarter)

# === Compute quarter-wise summary ===
quarter_summary = slouch_log_df.groupby('quarter').agg(
    total_events=('start_time', 'count'),
    max_lean=('max_lean_angle', 'max'),
    avg_lean_angle=('max_lean_angle', 'mean')
).reset_index()

# === Scoring function ===
def scale_score(value, ideal_max, worst_max):
    if pd.isna(value):
        return 100
    if value <= ideal_max:
        return 100
    elif value >= worst_max:
        return 0
    return 100 * (worst_max - value) / (worst_max - ideal_max)

# === Apply scoring ===
scores = []
for _, row in quarter_summary.iterrows():
    score_slouch = 0 if row['total_events'] > 0 else 100
    score_peak = scale_score(row['max_lean'], 20, 60)
    total_score = 0.4 * score_slouch + 0.6 * score_peak
    scores.append(total_score)

quarter_summary['quarter_score'] = scores

# === Add missing quarters ===
for q in ['Q1', 'Q2', 'Q3', 'Q4']:
    if q not in quarter_summary['quarter'].values:
        quarter_summary.loc[len(quarter_summary)] = [q, 0, 0.0, 0.0, 100.0]

# === Sort by Q1-Q4 order ===
quarter_summary = quarter_summary.set_index('quarter').reindex(['Q1', 'Q2', 'Q3', 'Q4']).reset_index()

# === Add total/average row ===
total_row = pd.DataFrame([{
    'quarter': 'Avg',
    'total_events': quarter_summary['total_events'].sum(),
    'max_lean': quarter_summary['max_lean'].max(),
    'avg_lean_angle': quarter_summary['avg_lean_angle'].mean(),
    'quarter_score': quarter_summary['quarter_score'].mean()
}])

quarter_summary = pd.concat([quarter_summary, total_row], ignore_index=True)

# === Plot ===
fig, ax = plt.subplots(figsize=(8, 5))
colors = ['skyblue'] * 4 + ['lightgreen']
bars = ax.bar(quarter_summary['quarter'], quarter_summary['quarter_score'], color=colors)

ax.set_ylim(0, 100)
ax.set_ylabel("Quarter Score")

# Add more space below the title
ax.set_title("Ergonomic Score by Quarter", pad=20)

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height + 2, f'{height:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


# === Save summary ===
summary_path = os.path.join(output_dir, "quarter_score_summary.csv")
quarter_summary.to_csv(summary_path, index=False)
print("✅ Saved to:", summary_path)

quarter_summary


✅ Saved to: /Users/moon/Documents/Posture/Bluetooth_dual_data/2025-05-30_[10-14_10-16]/quarter_score_summary.csv


,quarter,total_events,max_lean,avg_lean_angle,quarter_score
0,Q1,0,0.00000,0.000000,100.0
1,Q2,1,79.30427,79.304270,0.0
2,Q3,0,0.00000,0.000000,100.0
3,Q4,0,0.00000,0.000000,100.0
4,Avg,1,79.30427,19.826068,75.0


## **\*\*\***


## Old


In [97]:
import pandas as pd

# Define the pass/fail test data
data = {
    "Test Angle (°)": [15, 30, 45, 60, 90, 20, 10, 25, 40, 50],
    "Duration Held (sec)": [60] * 10,
    "Average Angle Detected (°)": [21, 35, 56, 62, 92, 30, 5, 31, 45, 51],
    "Detection Time (sec)": [61, 60, 62, 62, 88, 60, 57, 61, 58, 59],
    "Pass Criteria Met": ["Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes"],
    "Result (Pass/Fail)": ["Pass", "Pass", "Pass", "Pass", "Pass", "Pass", "Pass", "Pass", "Pass", "Pass"]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df

import pandas as pd

# Define the qualitative + quantitative posture detection results
qual_data = {
    "Trial #": [1, 2, 3, 4, 5],
    "Duration Held (sec)": [60 , 60, 60,60,60],
    "Observed Posture": ["Leaned", "Neutral", "Leaned", "Neutral", "Leaned"],
    "Average System Detection Angle (°)": [45,20,30,15,50],
    "Detection Time (sec)": [57,62,65,66,63],
    "Match?": ["Yes", "Yes", "No", "No", "Yes"],
    "Result (Pass/Fail)": ["Pass", "Fail", "Pass", "Fail", "Pass"]
}

# Create the DataFrame
qual_df = pd.DataFrame(qual_data)

# Display the DataFrame
qual_df



,Trial #,Duration Held (sec),Observed Posture,Average System Detection Angle (°),Detection Time (sec),Match?,Result (Pass/Fail)
0,1,60,Leaned,45,57,Yes,Pass
1,2,60,Neutral,20,62,Yes,Fail
2,3,60,Leaned,30,65,No,Pass
3,4,60,Neutral,15,66,No,Fail
4,5,60,Leaned,50,63,Yes,Pass


In [102]:
import numpy as np
# Simulate readings for Sensor 1 and Sensor 2
sensor_comparison_data = {
    "Test Angle (°)": [10, 10, 10, 15, 15, 15, 30, 30, 30, 45, 45, 45, 60, 60, 60],
    "Sensor 1 (°)": [11, 12, 10.5, 16, 14.5, 15.2, 28, 33, 30.5, 44.8, 45.5, 46, 59, 70, 65],
    "Sensor 2 (°)": [10.8, 11.5, 10.2, 15.8, 14.2, 15.0, 29.5, 32.5, 31.2, 45.2, 44.5, 45.6, 60.5, 68, 64]
}

# Create DataFrame
sensor_comparison_df = pd.DataFrame(sensor_comparison_data)

# Calculate deviations
sensor_comparison_df["Deviation Sensor 1 (°)"] = np.abs(sensor_comparison_df["Test Angle (°)"] - sensor_comparison_df["Sensor 1 (°)"])
sensor_comparison_df["Deviation Sensor 2 (°)"] = np.abs(sensor_comparison_df["Test Angle (°)"] - sensor_comparison_df["Sensor 2 (°)"])

# Determine pass/fail for each sensor
sensor_comparison_df["Sensor 1 Result"] = sensor_comparison_df["Deviation Sensor 1 (°)"].apply(lambda x: "Pass" if x <= 5 else "Fail")
sensor_comparison_df["Sensor 2 Result"] = sensor_comparison_df["Deviation Sensor 2 (°)"].apply(lambda x: "Pass" if x <= 5 else "Fail")

# Display the table
sensor_comparison_df


,Test Angle (°),Sensor 1 (°),Sensor 2 (°),Deviation Sensor 1 (°),Deviation Sensor 2 (°),Sensor 1 Result,Sensor 2 Result
0,10,11.0,10.8,1.0,0.8,Pass,Pass
1,10,12.0,11.5,2.0,1.5,Pass,Pass
2,10,10.5,10.2,0.5,0.2,Pass,Pass
3,15,16.0,15.8,1.0,0.8,Pass,Pass
4,15,14.5,14.2,0.5,0.8,Pass,Pass
5,15,15.2,15.0,0.2,0.0,Pass,Pass
6,30,28.0,29.5,2.0,0.5,Pass,Pass
7,30,33.0,32.5,3.0,2.5,Pass,Pass
8,30,30.5,31.2,0.5,1.2,Pass,Pass
9,45,44.8,45.2,0.2,0.2,Pass,Pass


In [ ]:
# Re-import required libraries after code execution state reset
import pandas as pd
import os
import matplotlib.pyplot as plt

# === Reload Data ===
angles_df = pd.read_csv(os.path.join(output_dir, "lean_angle_diff.csv"))
neck_df = pd.read_csv(os.path.join(output_dir, "neck_data.csv"))
pelvis_df = pd.read_csv(os.path.join(output_dir, "pelvis_data.csv"))
summary = pd.read_csv(os.path.join(output_dir, "todays_ergonomic_summary.csv"))

# Recompute seconds and merge pitch data
neck_df['timestamp'] = pd.to_datetime(neck_df['timestamp'])
pelvis_df['timestamp'] = pd.to_datetime(pelvis_df['timestamp'])
angles_df['timestamp'] = pd.to_datetime(angles_df['timestamp'])
angles_df['seconds'] = (angles_df['timestamp'] - angles_df['timestamp'].min()).dt.total_seconds()

neck_pitch = neck_df[['timestamp', 'pitch (deg)']].rename(columns={'pitch (deg)': 'neck_pitch'}).sort_values('timestamp')
pelvis_pitch = pelvis_df[['timestamp', 'pitch (deg)']].rename(columns={'pitch (deg)': 'pelvis_pitch'}).sort_values('timestamp')

# Merge pitch data into angles_df
angles_df = pd.merge_asof(
    angles_df.sort_values('timestamp'), 
    neck_pitch.sort_values('timestamp'), 
    on='timestamp', direction='nearest', tolerance=pd.Timedelta("50ms")
)
angles_df = pd.merge_asof(
    angles_df.sort_values('timestamp'), 
    pelvis_pitch.sort_values('timestamp'), 
    on='timestamp', direction='nearest', tolerance=pd.Timedelta("50ms")
)
angles_df.dropna(subset=['neck_pitch', 'pelvis_pitch'], inplace=True)

# Calculate quarter length
quarter_length = angles_df['seconds'].max() / 4

# === Plot 1: Leaning angle, neck pitch, pelvis pitch ===
import matplotlib.pyplot as plt

# Recalculate relative angles based on the first 10 seconds of neck and pelvis pitch
ref_df = angles_df[angles_df['seconds'] <= 10]
neck_ref = ref_df['neck_pitch'].mean()
pelvis_ref = ref_df['pelvis_pitch'].mean()

angles_df['rel_neck_pitch'] = angles_df['neck_pitch'] - neck_ref
angles_df['rel_pelvis_pitch'] = angles_df['pelvis_pitch'] - pelvis_ref
angles_df['normalized_lean_angle'] = angles_df['rel_neck_pitch'] - angles_df['rel_pelvis_pitch']

# Plot normalized lean angle
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(angles_df['seconds'], angles_df['normalized_lean_angle'], label='Normalized Lean Angle', color='blue')

# Horizontal threshold lines for ±10°
ax.axhline(y=10, color='red', linestyle=':', label='Slouch Threshold (+10°)')
ax.axhline(y=-10, color='red', linestyle=':')

# Vertical lines for quarters
for q in [1, 2, 3]:
    x = q * quarter_length
    ax.axvline(x=x, color='gray', linestyle='--')

ax.set_title("Normalized Lean Angle Over Time (Relative to First 10s)")
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Angle (°)")
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

# === Plot 2: Quarter Score Bar Chart ===
# Update the bar chart with different color for 'Total' bar
fig2, ax2 = plt.subplots(figsize=(8, 5))

# Color map: regular bars are skyblue, 'Total' is darkblue
colors = ['skyblue' if q != 'Total' else 'steelblue' for q in summary['quarter']]
bars = ax2.bar(summary['quarter'], summary['quarter_score'], color=colors)

# Add value labels
ax2.set_ylim(0, 100)
ax2.set_ylabel("Quarter Score")
ax2.set_title("Ergonomic Score by Quarter")
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, height + 2, f'{height:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import os

# === Load Data ===
output_dir = output_dir
#output_dir = "/Users/moon/Documents/Posture/Bluetooth_dual_data/2025-05-29_[22-13_22-18]"
neck_df = pd.read_csv(os.path.join(output_dir, "neck_data.csv"))
pelvis_df = pd.read_csv(os.path.join(output_dir, "pelvis_data.csv"))

# === Convert timestamp to datetime ===
neck_df['timestamp'] = pd.to_datetime(neck_df['timestamp'])
pelvis_df['timestamp'] = pd.to_datetime(pelvis_df['timestamp'])

# === Sort and rename for merging ===
neck_pitch = neck_df[['timestamp', 'pitch (deg)']].rename(columns={'pitch (deg)': 'neck_pitch'}).sort_values('timestamp')
pelvis_pitch = pelvis_df[['timestamp', 'pitch (deg)']].rename(columns={'pitch (deg)': 'pelvis_pitch'}).sort_values('timestamp')

# === Merge with tolerance of 50ms ===
angles_df = pd.merge_asof(
    neck_pitch, pelvis_pitch,
    on='timestamp', direction='nearest',
    tolerance=pd.Timedelta("50ms")
).dropna(subset=['pelvis_pitch'])

# === Compute time offset ===
angles_df['seconds'] = (angles_df['timestamp'] - angles_df['timestamp'].min()).dt.total_seconds()

# === Use first 10 seconds as reference ===
reference_duration = 10

ref_df = angles_df[angles_df['seconds'] <= reference_duration]
neck_ref = ref_df['neck_pitch'].mean()
pelvis_ref = ref_df['pelvis_pitch'].mean()

# === Normalize pitches relative to reference ===
angles_df['rel_neck_pitch'] = angles_df['neck_pitch'] - neck_ref
angles_df['rel_pelvis_pitch'] = angles_df['pelvis_pitch'] - pelvis_ref

# === Recalculate lean angle from relative pitch ===
angles_df['lean_angle'] = angles_df['rel_neck_pitch'] - angles_df['rel_pelvis_pitch']
angles_df['is_slouched'] = angles_df['lean_angle'].abs() > 10


# === Label Continuous Slouch Blocks ===
angles_df['slouch_block'] = 0
block_id = 1
slouch_duration = 0
in_slouch = False

for i, is_slouched in enumerate(angles_df['is_slouched']):
    if is_slouched:
        slouch_duration += 1
        if not in_slouch:
            block_start = i
            in_slouch = True
    else:
        if slouch_duration >= 60:
            angles_df.loc[block_start:i, 'slouch_block'] = block_id
            block_id += 1
        slouch_duration = 0
        in_slouch = False

slouch_duration = 20
# Catch end of last block
if in_slouch and slouch_duration >= slouch_duration:
    angles_df.loc[block_start:, 'slouch_block'] = block_id


# === Log Deviation Events ===
deviation_events = angles_df[angles_df['is_slouched']][['timestamp', 'lean_angle']]
deviation_events.to_csv(os.path.join(output_dir, "deviation_log.csv"), index=False)
total_deviation_events = len(deviation_events)
print(f"Total number of deviations >10°: {total_deviation_events}")

# === Divide into 4 quarters ===
total_time = angles_df['seconds'].max()
quarter_length = total_time / 4
angles_df['quarter'] = pd.cut(
    angles_df['seconds'],
    bins=[-1, quarter_length, 2*quarter_length, 3*quarter_length, total_time + 1],
    labels=['Q1', 'Q2', 'Q3', 'Q4']
)

# === Count deviations per quarter ===
deviation_counts = angles_df[angles_df['is_slouched']].groupby('quarter').size().reset_index(name='deviation_count')

# === Summary Calculations ===
summary = angles_df.groupby('quarter').agg(
    slouched_pct=('is_slouched', lambda x: 100 * x.sum() / len(x)),
    peak_flexion=('lean_angle', lambda x: x.abs().max())
).reset_index()

# === Compute avg_deviation only from long slouch blocks
avg_devs = []
for q in summary['quarter']:
    mask = (angles_df['quarter'] == q) & (angles_df['slouch_block'] > 0)
    if mask.any():
        avg_devs.append(angles_df.loc[mask, 'lean_angle'].abs().mean())
    else:
        avg_devs.append(0.0)
summary['avg_deviation'] = avg_devs

# === Merge deviation counts into summary ===
summary = pd.merge(summary, deviation_counts, on='quarter', how='left')
summary['deviation_count'] = summary['deviation_count'].fillna(0).astype(int)

# === Score Scaling Function ===
def scale_score(value, ideal_max, worst_max):
    if value <= ideal_max:
        return 100
    elif value >= worst_max:
        return 0
    else:
        return 100 * (worst_max - value) / (worst_max - ideal_max)

# === Compute Scores ===
scores = []
for quarter in summary['quarter']:
    quarter_data = angles_df[angles_df['quarter'] == quarter]
    
    # Check if there is any slouching period ≥ 60s
    has_long_slouch = quarter_data['slouch_block'].gt(0).any()
    score_slouch = 0 if has_long_slouch else 100

    row = summary[summary['quarter'] == quarter].iloc[0]
    score_peak = scale_score(row['peak_flexion'], 20, 60)
    score_dev = scale_score(row['avg_deviation'], 10, 40)

    total_score = (
        0.4 * score_slouch +
        0.3 * score_peak +
        0.3 * score_dev
    )
    scores.append(total_score)

summary['quarter_score'] = scores

# === Add Total Row ===
overall = summary[['slouched_pct', 'peak_flexion', 'avg_deviation', 'deviation_count']].mean()
score_slouch = 0 if any(angles_df['slouch_block'] > 0) else 100
score_peak = scale_score(overall['peak_flexion'], 20, 60)
score_dev = scale_score(overall['avg_deviation'], 10, 40)
total_score = (
    0.4 * score_slouch +
    0.3 * score_peak +
    0.3 * score_dev
)

summary.loc[len(summary)] = ['Total', *overall.values, total_score]

# === Save Summary ===
summary.to_csv(os.path.join(output_dir, "todays_ergonomic_summary.csv"), index=False)

# === Final Output ===
summary
